In [1]:
#Import python libraries
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [2]:
#Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\10pai\.wdm\drivers\chromedriver\win32\91.0.4472.19\chromedriver.exe] found in cache


In [3]:
#Visit mars.nasa.gov website
nasa_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(nasa_url)

#Add sleep to allow page to load
time.sleep(2)

#Create Beautiful Soup object to scrape most recent article
html = browser.html
soup = bs(html, 'html.parser')

In [4]:
#Scrape most recent article title and description on mars from Nasa
articles = soup.find_all('div', class_='list_text')[0]
news_title = articles.find(class_='content_title').text
news_p = articles.find(class_='article_teaser_body').text

In [5]:
#Store scraped data in dictionary
mars_dict = {
    "news_title" : news_title,
    "news_p": news_p
}

In [6]:
#Shutdown splinter instance
browser.quit()

In [7]:
#Set-up splinter instance
browser = Browser('chrome', **executable_path, headless=False)

#Visit data-class-jpl website
JPL_url= 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(JPL_url)

#Create Beautiful Soup object to scrape JPL images
html = browser.html
soup = bs(html, 'html.parser')

In [8]:
#Scrape featured image URL
feature_image = soup.find('div', class_='floating_text_area')
a = feature_image.find('a')
href = a['href']
featured_image_url = (f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{href}')

In [9]:
#Add feature image URL to mars.dict dictionary
mars_dict['featured_image_url'] = featured_image_url

In [10]:
#Shutdown splinter instance
browser.quit()

In [11]:
#URL to scrape tables containing Mars data
facts_url = 'https://space-facts.com/mars/'

#Scrape tables from facts_url using pandas
tables = pd.read_html(facts_url)

In [12]:
#Convert first table to pandas dataframe
mars_df = tables[0]

#Rename columns of dataframe and reset index to Characteristic column
mars_df.rename(columns={ 0: 'Characteristic', 1: "Mars"}, inplace=True)
mars_df.set_index('Characteristic', inplace=True)

#Generate html table
mars_html_table = mars_df.to_html()

#Add to mars_dict
mars_dict['mars facts'] = mars_html_table

In [13]:
#Set-up splinter instance
browser = Browser('chrome', **executable_path, headless=False)

#Visit astrogeology.usgs.gov website
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemi_url)

#Create Beautiful Soup object to scrape JPL images
html = browser.html
soup = bs(html, 'html.parser')
results = soup.find_all('div', class_='description')


#Create list to store hemi URLs
mars_hemi_image_urls = []

#Loop through each of the four hemisphere pages
for hemis in results:
    hemi = hemis.find('h3').text
    browser.click_link_by_partial_text(hemi)
    html = browser.html
    soup = bs(html, 'html.parser')
    hemi_images = soup.find('div', class_='downloads')
    hemi_a = hemi_images.find('a')
    hemi_href = hemi_a['href']
    hemi_dict = {
        'title' : hemi,
        'img_url' : hemi_href
    }
    mars_hemi_image_urls.append(hemi_dict)
    browser.back()

C:\Users\10pai\anaconda3\envs\PythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:493: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [14]:
#Add hemi image urls to mars_dict
mars_dict['mars_hemi_image_urls'] = mars_hemi_image_urls

In [15]:
#Shutdown splinter instance
browser.quit()